In [1]:
import gymnasium as gym
import numpy as np
import random
from gymnasium.wrappers import RecordVideo
from IPython.display import HTML
from base64 import b64encode

In [2]:
def test_policy_video(policy_table, name, video_width=500):

    video_path = './video/' + name + '-step-0.mp4'
        
    env_test = gym.make('Taxi-v3', render_mode='rgb_array')
    env_test = RecordVideo(env=env_test, video_folder="./video", name_prefix=name, step_trigger=lambda x : True, disable_logger=True)

    state, info = env_test.reset()
    done = False
    steps = 0
    
    env_test.start_video_recorder()

    while not done:
        action = policy_table[state]
        new_state, reward, done, truncated, info = env_test.step(int(action))
        state = new_state
        env_test.render()
        steps += 1
        if steps > 50:
            break
        
    env_test.close_video_recorder()

    env_test.close()

    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

## Defining Taxi Environment

In [3]:
class Taxi:
    def __init__(self,
            learning_rate=0.9, 
            discount_rate=0.8,
            epsilon=1.0,
            decay_rate=0.005,
            num_iter=200,
            num_episodes=10000,
            max_steps=99,
            num_evaluate_steps = 20):
        self.env = gym.make('Taxi-v3') #initializing the environment
        self.state_size = self.env.observation_space.n #size of state space (it is 500 in taxi environment)
        self.action_size = self.env.action_space.n #size of action space (it is 6 in taxi environemt)
        self.vtable = np.zeros(self.state_size) #initializing state value table
        self.ptable = np.zeros(self.state_size) #initializing policy table (each entry contains one of 6 aciton)
        self.qtable = np.zeros((self.state_size, self.action_size)) #initializing q value table (value for each (state, action) pair)
        self.learning_rate = learning_rate #learning rate used in Q-leanring algorithm
        self.discount_rate = discount_rate #discount factor (gamma)
        self.epsilon = epsilon #epsilon for specifying randomness threshold in epsilon-greedy action used in Q-learning
        self.decay_rate = decay_rate #decay rate for decreasing the epsilon during the algorithm
        self.num_iter = num_iter #number of iteration used in algorihtm
        self.num_episodes = num_episodes #number of episodes (each episode is a sequence from start to end of a single game)
        self.max_steps = max_steps #maximum bound for number of steps in each episode
        self.num_evaluate_steps = num_evaluate_steps #number of steps to evaluate a given policy (just used in policy iteration algorithm)
        
    
                

## 1. Value Iteration

### 1.1. Initializing Parameters

In [4]:
#TODO: Initialize the parameters by filling the blanks
taxi = Taxi(num_iter=500, num_evaluate_steps=50, discount_rate=0.9)

### 1.2. Finding Optimal State Values

In [5]:
# you can use taxi.state_size and taxi.action_size
# you can also use taxi.env.P[state][action] which returns a tuple containing (probability, next state, reward, if it is done or not)
def value_iteration(vtable, num_iter, discount_rate):
    for _ in range(num_iter):
        v_old = np.copy(vtable)
        for state in range(taxi.state_size) :
            temp_action = np.zeros(taxi.action_size)
            #TODO: compute the value for each action and fill the temp_action array with those values
            for action in range(taxi.action_size):
                for prob, next_state, reward, done in taxi.env.P[state][action]:
                    temp_action[action] += prob * (reward + discount_rate * v_old[next_state])
            vtable[state] = np.max(temp_action) #assigning the best action value to the state
        

### 1.3. Extracting The Optimal Policy

In [6]:
def optimal_policy_extraction(vtable, ptable, num_iter, discount_rate):
    value_iteration(vtable, num_iter, discount_rate)
    for state in range(taxi.state_size) :
        temp_action = np.zeros(taxi.action_size)
        #TODO: compute the value for each action and fill the temp_action array with those values
        for action in range(taxi.action_size):
            for prob, next_state, reward, done in taxi.env.P[state][action]:
                temp_action[action] += prob * (reward + discount_rate * vtable[next_state])
        ptable[state] = np.argmax(temp_action) #finding the best action by argmax

### 1.4. Running The Algorithm

In [7]:
optimal_policy_extraction(taxi.vtable, taxi.ptable, taxi.num_iter, taxi.discount_rate)
optimal_policy = taxi.ptable.copy()     # saving determined optimal policy

c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


### 1.5. Tesing

In [8]:
test_policy_video(taxi.ptable, 'value-iteration')

c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\venv\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## 2. Policy Iteration

### 2.1. Initialize Parameters

In [9]:
#TODO: Initialize the parameters by filling the blanks
taxi = Taxi(num_iter=500, num_evaluate_steps=50, discount_rate=0.9)

### 2.2. Policy Evaluation

In [10]:
def evaluate(num_iter, discount_rate):
    vtable = np.zeros(taxi.state_size)
    for _ in range(num_iter):
        v_old = np.copy(vtable)
        for state in range(taxi.state_size):
            #TODO: compute the value of the state according to the current policy
            action = taxi.ptable[state]
            vtable[state] = sum(prob * (reward + discount_rate * v_old[next_state])
                                for prob, next_state, reward, done in taxi.env.P[state][action])
    return vtable

### 2.3. Policy Improvement

In [11]:
def improvement(ptable, num_iter, num_evaluate_steps, discount_rate):
    for _ in range(num_iter):
        vtable = evaluate(num_evaluate_steps, discount_rate).copy()
        for state in range(taxi.state_size) :
            temp_action = np.zeros(taxi.action_size)
            #TODO: compute the value for each action and fill the temp_action array with those values
            for action in range(taxi.action_size):
                for prob, next_state, reward, done in taxi.env.P[state][action]:
                    temp_action[action] += prob * (reward + discount_rate * vtable[next_state])
            ptable[state] = np.argmax(temp_action) #improving the policy

### 2.4. Running The Algorithm

In [12]:
improvement(taxi.ptable, taxi.num_iter, taxi.num_evaluate_steps, taxi.discount_rate)

c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


### 2.5. Testing

In [13]:
test_policy_video(taxi.ptable, 'policy-iteration')

c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\venv\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\KIANA PEIRO\Desktop\uni\terme 10 (cam)\computer\Principles & Applications of Artificial Intelligence\Project\Project 3\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## 3. Q-learning

### 3.1. Initializing Parameters

In [14]:
#TODO: Initialize the parameters by filling the blanks
taxi = Taxi(num_episodes=10000, max_steps=99, learning_rate=0.9, discount_rate=0.9, epsilon=1.0, decay_rate=0.005)

### 3.2. Training

In [15]:

def q_learing_train(qtable, num_episodes, max_steps, learning_rate, discount_rate, epsilon, decay_rate):
    for episode in range(num_episodes):
        state, info = taxi.env.reset()
        done = False
        for s in range(max_steps):
            #epsilon greedy
            if random.uniform(0,1) < epsilon:
                action = taxi.env.action_space.sample()
            else:
                action = np.argmax(qtable[state, :]) #TODO: assign the action for greedy case
            new_state, reward, done, truncated, info = taxi.env.step(action) #doing one step
            qtable[state][action] = qtable[state][action] + learning_rate * (
                reward + discount_rate * np.max(qtable[new_state, :]) - qtable[state][action]
            ) #TODO: update q-value (main part of Q-learning)
            state = new_state
            if done == True:
                break
        epsilon = np.exp(-decay_rate*episode)

### 3.3. Runing The Algorithm

In [16]:
q_learing_train(taxi.qtable, taxi.num_episodes, taxi.max_steps, taxi.learning_rate, taxi.discount_rate, taxi.epsilon, taxi.decay_rate)

### 3.4. Etracting The Policy

In [17]:
for state in range(taxi.state_size):
    taxi.ptable[state] = np.argmax(taxi.qtable[state][:])

### 3.5. Testing

In [18]:
test_policy_video(taxi.ptable, 'Q-learning')

## 4. Direct Evaluation(Monte Carlo)

In this section, the policy is given and you should just determine the value of each state based on the given policy. Then you can evaluate states, based on the optimal policy (you found in the previous sections) and then compare the 2 matrices.

### 4.1. Initializing Parameters

In [19]:
#TODO: Initialize the parameters by filling the blanks
taxi = Taxi(num_episodes=10000, max_steps=99, discount_rate=0.9)

### 4.2. Training

In [20]:
def monte_carlo(ptable, num_episodes, max_steps, discount_rate):
    count = np.ones(taxi.state_size) # counts the number of visits to each state
    vtable = np.zeros(taxi.state_size) # store the total return for each state
    for _ in range(num_episodes):
        state, info = taxi.env.reset()
        done = False
        trajectory = [state]
        rewards = []
        for s in range(max_steps):
            action = ptable[state]
            new_state, reward, done, truncated, info = taxi.env.step(action)
            trajectory.append(new_state)
            #TODO For each state visited in the current episode, update the value function vtable using the Monte Carlo update formula
            rewards.append(reward)
            count[new_state] += 1
            state = new_state
            if done == True:
                break
        G = 0
        for t in reversed(range(len(trajectory))):
            state = trajectory[t]
            G = reward + discount_rate * G
            if state not in trajectory[:t]:  # First visit Monte Carlo
                vtable[state] += G
                count[state] += 1
    vtable /= count
    return vtable

In [21]:
given_policy = np.zeros(taxi.state_size)
given_policy_state_values = monte_carlo(given_policy, taxi.num_episodes, taxi.max_steps, taxi.discount_rate)

In [22]:
optimal_policy_state_values = monte_carlo(optimal_policy, taxi.num_episodes, taxi.max_steps, taxi.discount_rate)

### 4.3. Testing

In [23]:
print(optimal_policy_state_values - given_policy_state_values)

[  9.99793687  79.61326144  62.74794669  76.00042442 178.50692462
   0.         177.47764108 168.04582466 143.44843206 163.90028597
   0.         158.50479697 172.29404981 164.45504496 173.43605663
   0.          18.99608005  65.09386548  46.82710104  61.21987904
   0.          82.8641808   68.00493959  80.15079845 174.49654124
   0.         176.02241571 164.76226677 148.97025229 168.74732471
   0.         164.25876451 166.11872451 160.39587264 167.81366506
   0.          27.0916125    0.           0.           0.
   0.         170.77802473 153.91255797 168.67108428  86.29761105
   0.          86.70257977  77.197449   152.97369416 171.34403936
   0.         166.89279262 164.55685221 154.38593306 163.26429794
   0.           0.          34.37975573   0.           0.
   0.         174.59675884 159.15568591 171.32524012  83.27975285
   0.          83.03219316  72.12894108 158.17618592 174.83849855
   0.         171.02580933 158.58248915 148.2424932  158.87046929
   0.           0.        